<a href="https://colab.research.google.com/github/bikash119/learn_tensorflow/blob/main/learn_tf_004onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Data

In [2]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip

#unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_all_data.zip")
zip_ref.extractall()
zip_ref.close

--2023-07-09 13:42:54--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.128, 172.253.117.128, 142.250.99.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519183241 (495M) [application/zip]
Saving to: ‘10_food_classes_all_data.zip’

10_food_classes_all 100%[===================>] 495.13M   227MB/s    in 2.2s    

2023-07-09 13:42:56 (227 MB/s) - ‘10_food_classes_all_data.zip’ saved [519183241/519183241]



<bound method ZipFile.close of <zipfile.ZipFile filename='10_food_classes_all_data.zip' mode='r'>>

In [3]:
!rm -rf /content/__MACOSX

In [4]:
from typing import List
from pathlib import Path
def get_classnames(train_dir) ->List[str]:
  """
    Read a directory containing training data which has a folder structure of a multi-class classification and
    returns a list of folder names a.k.a classes
    Args:
      data_dir(str): absolute path of the directory
    Returns:
      classnames(List[str]): List of string
  """

  data_dir = Path(train_dir)
  classnames = sorted([item.name for item in data_dir.glob("*")])
  return classnames


In [6]:
classnames = get_classnames("/content/10_food_classes_all_data/train")
classnames

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [7]:
train_dir = Path("/content/10_food_classes_all_data/train")
test_dir = Path("/content/10_food_classes_all_data/test")
train_dir, test_dir

(PosixPath('/content/10_food_classes_all_data/train'),
 PosixPath('/content/10_food_classes_all_data/test'))

In [9]:
import tensorflow as tf

## Create train and test datasets
train_dataset = tf.keras.utils.image_dataset_from_directory(directory=train_dir
                                                             ,labels="inferred"
                                                             ,label_mode="categorical"
                                                             ,image_size=(224,224)
                                                             ,batch_size=32
                                                             ,shuffle=True)

test_dataset = tf.keras.utils.image_dataset_from_directory(directory=test_dir
                                                             ,labels="inferred"
                                                             ,label_mode="categorical"
                                                             ,image_size=(224,224)
                                                             ,batch_size=32
                                                             ,shuffle=True)

Found 7500 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [13]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [14]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
# Set seed
tf.random.set_seed(42)

# build model
model_1 = Sequential([
    layers.Rescaling(1./255,input_shape=(224,224,3))
    ,layers.Conv2D(10,3,activation="relu")
    ,layers.MaxPool2D(pool_size=2)
    ,layers.Conv2D(10,3,activation="relu")
    ,layers.MaxPool2D()
    ,layers.Conv2D(10,3,activation="relu")
    ,layers.MaxPool2D()
    ,layers.Flatten()
    ,layers.Dense(10,activation="sigmoid")
])

# compile the model
model_1.compile(loss=tf.keras.losses.categorical_crossentropy
                ,optimizer=tf.keras.optimizers.Adam()
                ,metrics=["accuracy"]
                )
# fit the model
history_1 = model_1.fit(train_dataset
                        ,steps_per_epoch=len(train_dataset)
                        ,validation_data=test_dataset
                        ,validation_steps=len(test_dataset)
                        ,epochs=5)

Epoch 1/5
 78/235 [========>.....................] - ETA: 2:41 - loss: 2.2705 - accuracy: 0.1346

KeyboardInterrupt: ignored